In [49]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re


BASE_URL = 'https://www.nationalheraldindia.com'
country = 'bangladesh'
initial_url = f'{BASE_URL}/search?q={country}'

#COLLECTING THE NEWS LINKS 

driver = webdriver.Chrome()
driver.get(initial_url)

time.sleep(10)

load_more_button = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div/div[1]/div/button')

#ADJUST HOW MANY TIMES 'LOAD MORE' BUTTON SHOULD BE CLICKED

number_of_times_load_button_clicks = 1

while number_of_times_load_button_clicks > 0:

    driver.execute_script("arguments[0].scrollIntoView();", load_more_button)

    time.sleep(5)

    ActionChains(driver)\
    .click(load_more_button)\
    .perform()

    time.sleep(5)
    
    number_of_times_load_button_clicks -= 1

soup = BeautifulSoup(driver.page_source, 'html.parser')

main_div = soup.find('div', {'id':'container'})

all_news_links = []

if main_div:

    all_link_divs = main_div.find_all('a', class_ = 'styles-m__story-card-large__1wVEJ')

    for each_link_div in all_link_divs:

        all_news_links.append(each_link_div.get('href'))
else:
    
    print('Error in page')

driver.quit()

#SCRAPING THE NEWS LINKS

counter = 0
data_list = []

#SCRAPING THE FIRST 10 LINKS

for link in all_news_links[:10]:

        url = f'{BASE_URL}{link}'

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        #TITLE

        title_tag = soup.find('div', class_= 'styles-m__title-wrapper__2qBP_').find('h1')
        title = title_tag.text if title_tag else 'Title not found'
        title_translation = 'None'

        #Content Summary

        content_summary_tag = soup.find('div', class_= 'styles-m__title-wrapper__2qBP_').find('h2')
        content_summary = content_summary_tag.text if content_summary_tag else 'Content summary not found'
        summary_translation = 'None'

        #Author

        author_tag = soup.find('h4', class_ = 'styles-m__author-name__1hX0n')
        author = author_tag.text if author_tag else 'Author not found'

        #Date

        date_info = soup.find('time', class_ = 'styles-m__story-published-date__JqZcx')

        if date_info:

            date_data = date_info.get('datetime')

            only_date = date_data.split('T')[0]
            only_time = date_data.split('T')[1]
            time_formatted = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
            cleaned_date = f"{only_date},{time_formatted}"

            source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M")
            bangladesh_localtime = source_localtime  + timedelta(minutes=30)

        else:

            date_data = 'Date data not found'

        #CONTENT

        content = []

        main_content_divs = soup.find_all('div', class_ = 'story-page-card')

        if len(main_content_divs) > 0:

            for each_div in main_content_divs:

                all_paras = each_div.find_all('p')

                for each_para in all_paras:

                    content.append(each_para.text)

            full_content = ''.join(content)
            full_content = full_content.replace('\xa0',' ')

            temp_cut = full_content.split('Follow us on')

            full_content = temp_cut[0]

        else:

            full_content = 'Content not found'

        content_translation = 'None'
        
        data_dict = {
            "url": url,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }

        counter+=1


        if (full_content != "Content Not Found" and title != 'Title not found'):

                if data_dict not in data_list:
                        # Adding to data list
                        data_list.append(data_dict)
                        print(f'Link {counter} added')
        else:
                print(f'Link {counter}')
                print('Skipped due to missing info.')
                

                
df = pd.DataFrame(data_list)
df.head()

csv_filename = f"{country}_National_Herald.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["bangladesh_localtime"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="date", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)


Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://www.nationalheraldindia.com/internatio...,How is Myanmar's civil war impacting Bangladesh?,Fighting between Myanmar's junta and the Araka...,Fighting between Myanmar's junta and rebel gro...,None,None,None,DW,bangladesh,2024-02-23 10:33:00,2024-02-23 11:03:00
1,https://www.nationalheraldindia.com/politics/w...,Why Modi is welcoming the Bangladesh election ...,Prime Minister Sheikh Hasina and her Awami Lea...,Sheikh Hasina winning a 4th consecutive term a...,None,None,None,DW,bangladesh,2024-01-12 21:51:00,2024-01-12 22:21:00
2,https://www.nationalheraldindia.com/internatio...,Bangladesh election: How real is women's influ...,"For the past few decades, #Bangladesh has been...","For decades, Bangladesh has been led by two wo...",None,None,None,DW,bangladesh,2024-01-12 20:39:00,2024-01-12 21:09:00
3,https://www.nationalheraldindia.com/national/b...,Bangladesh poll verdict a double-whammy for India,"Sheikh Hasina's expression of gratitude, thank...",India's predictable endorsement of the poll ve...,None,None,None,Subir Bhaumik,bangladesh,2024-01-09 14:16:00,2024-01-09 14:46:00
4,https://www.nationalheraldindia.com/internatio...,"US says Bangladesh elections not free or fair,...",A day after Bangladesh Prime Minister Sheikh H...,The US calls for an investigation into reports...,None,None,None,PTI,bangladesh,2024-01-09 08:51:00,2024-01-09 09:21:00
